# Segmentation Using Earth Engine Cloud

In [1]:
# Import libraries
import geemap
import ee
import pandas as pd

In [2]:
# # Connect to earth engine
# ee.Authenticate()

In [3]:
# Initialize ee
ee.Initialize()

In [4]:
# Define constants
REGIONS = ["1-bpm24", "1-gt1", "1-rric112", "2-bpm24", "2-gt1", "2-rric112"]
BANDS = ["rgb", "red", "green", "blue", "nir", "re"]
TIPE = ["gulma", "karet"]
ROOT = "/home/iqballm09/Documents/skripsi"
LOCATION = "projects/severity-karet-s1/assets/ortho"
DESTINATION = "projects/severity-karet-s1/assets/segmentation"

In [5]:
# Define location coordinate
loc_geo = pd.read_csv(ROOT + "/data/coordinates/locations.csv")
loc_geo.set_index("location", inplace=True)
loc_geo.head()

,latitude,longitude
location,,
rric112,-2.953957,104.509275
gt1,-2.950331,104.519218
bpm24,-2.924831,104.541644


## Delete Segmentation Data

In [6]:
# for region in REGIONS:
#     for tipe in TIPE:
#         for band in BANDS:
#             time = region.split("-")[0]
#             ee.data.deleteAsset('projects/severity-karet-s1/assets/segmentation/{}/{}/{}-{}'.format(region, tipe, time, band))

## Create ROI

In [7]:
# Function for create roi
def create_roi(fileName):
    coord = pd.read_csv(ROOT + "/data/coordinates/{}.csv".format(fileName))
    coord = coord[["Lon", "Lat"]]
    coord_mat = coord.values.tolist()
    roi_img = ee.Geometry.Polygon(coord_mat)
    return roi_img

In [8]:
# Extract roi
roi_1bpm24 = create_roi('1-bpm24')
roi_1gt1 = create_roi('1-gt1')
roi_1rric112 = create_roi('1-rric112')
roi_2bpm24 = create_roi('2-bpm24')
roi_2gt1 = create_roi('2-gt1')
roi_2rric112 = create_roi('2-rric112')

In [9]:
type(roi_2rric112)

ee.geometry.Geometry

## Clusters Analysis

Tujuan : Menganalisis segmen/cluster yang merupakan objek gulma atau perkebunan karet.

In [10]:
# Function for image clipping 
def clipping(roi, dirname, band):
    time = dirname.split("-")[0]
    path_ee = LOCATION + "/{}/{}-{}".format(dirname, time, band)
    output = ee.Image(path_ee).clip(roi)
    return output

In [11]:
# Funcation for image segmentation
def segmentation(img, roi, dirname, num_clusters=5):
    train_data = img.sample(**{"region":roi, "scale": 10, "numPixels": 1000})
    # Clustering process
    kmeans = ee.Clusterer.wekaKMeans(**{'nClusters': num_clusters, 'init':1}).train(train_data)
    cluster_result = img.cluster(kmeans)
    # Create segmentation result
    list_segments = []
    for i in range(num_clusters):
        cluster = cluster_result.select('cluster').eq(i)
        segment = img.updateMask(cluster)
        list_segments.append(segment)
    return cluster_result, list_segments

In [12]:
def show_geomap(img_segments, region):
    # Create map
    loc = region.split("-")[1]
    Map = geemap.Map(center=loc_geo.loc[loc].tolist(), zoom=200)
    for i in range(len(img_segments)):
        Map.addLayer(img_segments[i], {}, "segment {}".format(i))
    return Map

### 1-BPM24

In [13]:
# Segmentation
clipped_1bpm24 = clipping(roi_1bpm24, "1-bpm24", "rgb")
clusters_1bpm24, segments_1bpm24 = segmentation(clipped_1bpm24, roi_1bpm24, "1-bpm24")

In [14]:
# Show map
map_1bpm24 = show_geomap(segments_1bpm24, "1-bpm24")
map_1bpm24

Map(center=[-2.924831, 104.541644], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

Gulma --> 3\
Karet --> 0, 1, 2, 4

### 1-GT1

In [15]:
# Segmentation
clipped_1gt1 = clipping(roi_1gt1, "1-gt1", "rgb")
clusters_1gt1, segments_1gt1 = segmentation(clipped_1gt1, roi_1gt1, "1-gt1")

In [16]:
# Show map
map_1gt1 = show_geomap(segments_1gt1, "1-gt1")
map_1gt1

Map(center=[-2.950331, 104.519218], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

Gulma --> 1\
Karet --> 0, 2, 3, 4

### 1-RRIC112

In [17]:
# Segmentation
clipped_1rric112 = clipping(roi_1rric112, "1-rric112", "rgb")
clusters_1rric112, segments_1rric112 = segmentation(clipped_1rric112, roi_1rric112, "1-rric112")

In [18]:
map_1rric112 = show_geomap(segments_1rric112, "1-rric112")
map_1rric112

Map(center=[-2.953957, 104.509275], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

Gulma --> 1\
Karet --> 0, 2, 3, 4

### 2-BPM24

In [19]:
# Segmentation
clipped_2bpm24 = clipping(roi_2bpm24, "2-bpm24", "rgb")
clusters_2bpm24, segments_2bpm24 = segmentation(clipped_2bpm24, roi_2bpm24, "2-bpm24")

In [20]:
map_2bpm24 = show_geomap(segments_2bpm24, "2-bpm24")
map_2bpm24

Map(center=[-2.924831, 104.541644], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

Gulma --> 0\
Karet --> 1, 2, 3, 4

### 2-GT1

In [21]:
# Segmentation
clipped_2gt1 = clipping(roi_2gt1, "2-gt1", "rgb")
clusters_2gt1, segments_2gt1 = segmentation(clipped_2gt1, roi_2gt1, "2-gt1")

In [22]:
map_2gt1 = show_geomap(segments_2gt1, "2-gt1")
map_2gt1

Map(center=[-2.950331, 104.519218], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

Gulma --> 1\
Karet --> 0, 2, 3, 4

### 2-RRIC112

In [23]:
# Segmentation
clipped_2rric112 = clipping(roi_2rric112, "2-rric112", "rgb")
clusters_2rric112, segments_2rric112 = segmentation(clipped_2rric112, roi_2rric112, "2-rric112")

In [24]:
map_2rric112 = show_geomap(segments_2rric112, "2-rric112")
map_2rric112

Map(center=[-2.953957, 104.509275], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

Ada objek2 yang tidak tersegmentasi dengan baik sehingga hasil clusteringnya kurang bagus.

Gulma --> 1\
Karet --> 0, 2, 3, 4

## Segmentation

### Segmentation Process --> Exclude 2-RRIC112

In [25]:
# Function for segmentation all bands
def segmentation_process(img_clusters, roi, region_name, from_karet, from_gulma):
    to_karet = [1, 1, 1, 1]
    to_gulma = [1]

    mask_karet = img_clusters.remap(from_karet, to_karet, 0, 'cluster')
    mask_gulma = img_clusters.remap(from_gulma, to_gulma, 0, 'cluster')

    dict_karet = {}
    dict_gulma = {}

    for band in BANDS:
        img_clipped = clipping(roi, region_name, band)
        karet = img_clipped.updateMask(mask_karet)
        gulma = img_clipped.updateMask(mask_gulma)
        dict_karet[band] = karet
        dict_gulma[band] = gulma
    
    return dict_karet, dict_gulma

In [26]:
# Segmentation
karet_1bpm24, gulma_1bpm24 = segmentation_process(clusters_1bpm24, roi_1bpm24, "1-bpm24", [0, 1, 2, 4], [3])
karet_1gt1, gulma_1gt1 = segmentation_process(clusters_1gt1, roi_1gt1, "1-gt1", [0, 2, 3, 4], [1])
karet_1rric112, gulma_1rric112 = segmentation_process(clusters_1rric112, roi_1rric112, "1-rric112", [0, 2, 3, 4], [1])
karet_2bpm24, gulma_2bpm24 = segmentation_process(clusters_2bpm24, roi_2bpm24, "2-bpm24", [1, 2, 3, 4], [0])
karet_2gt1, gulma_2gt1 = segmentation_process(clusters_2gt1, roi_2gt1, "2-gt1", [0, 2, 3, 4], [1])

### Segmentation Process --> 2-RRIC112

In [27]:
# Function for segmentation specific band
def segmentation_process1(img_clusters, roi, region_name, band, from_karet, from_gulma):
    to_karet = [1, 1, 1, 1]
    to_gulma = [1]

    mask_karet = img_clusters.remap(from_karet, to_karet, 0, 'cluster')
    mask_gulma = img_clusters.remap(from_gulma, to_gulma, 0, 'cluster')

    dict_karet = {}
    dict_gulma = {}

    img_clipped = clipping(roi, region_name, band)
    karet = img_clipped.updateMask(mask_karet)
    gulma = img_clipped.updateMask(mask_gulma)
    dict_karet[band] = karet
    dict_gulma[band] = gulma
    
    return dict_karet, dict_gulma

In [28]:
# Segmentation
karet_2rric112, gulma_2rric112 = segmentation_process1(clusters_2rric112, roi_2rric112, "2-rric112", "rgb", [0, 2, 3, 4], [1])

### Show Results

In [29]:
# Function to show segmentation result on map
def show_map_results(region, imgs_karet, imgs_gulma):
    loc = region.split("-")[1]
    Map = geemap.Map(center=loc_geo.loc[loc].tolist(), zoom=200)
    Map.addLayer(imgs_karet["rgb"], {}, "segment karet")
    Map.addLayer(imgs_gulma["rgb"], {}, "segment gulma")
    return Map

In [30]:
res_1bpm24 = show_map_results("1-bpm24", karet_1bpm24, gulma_1bpm24)
res_1gt1 = show_map_results("1-gt1", karet_1gt1, gulma_1gt1)
res_1rric112 = show_map_results("1-rric112", karet_1rric112, gulma_1rric112)
res_2bpm24 = show_map_results("2-bpm24", karet_2bpm24, gulma_2bpm24)
res_2gt1 = show_map_results("2-gt1", karet_2gt1, gulma_2gt1)
res_2rric112 = show_map_results("2-rric112", karet_2rric112, gulma_2rric112)

In [41]:
res_2rric112

Map(bottom=2182741651.0, center=[-2.953957, 104.509275], controls=(WidgetControl(options=['position', 'transpa…

## Export Results

Upload hasil segmentasi ke Google Drive.

In [32]:
# Function for export images to drive all bands
def export_to_drive(imgs, roi, region, tipe):
    print("%s %s images being export to drive" % (region, tipe))
    print("------------------------------------------")
    time = region.split("-")[0]
    for band in BANDS:
        # Create task
        task = ee.batch.Export.image.toDrive(
            image=imgs[band],
            region=roi,
            description=band + "_" + region,
            folder=region + "/" + tipe,
            fileNamePrefix=time + "-" + band,
            fileFormat='GeoTIFF',
            scale=0.04,
            crs='EPSG:4326'
        )
        task.start()
    
        # Wait for respond
        task_status = task.status()
        while task_status['state'] in ['READY', 'RUNNING']:
            task_status = task.status()

        # Check if successful
        if task_status['state'] == 'COMPLETED':
            print("{} image export to drive success!".format(band))
        else:
            print('{} image export to drive failed!'.format(band))

In [33]:
# Function for export images to drive specific bands
def export_to_drive1(img, roi, region, tipe, band):
    print("%s %s images being export to drive" % (region, tipe))
    print("------------------------------------------")
    time = region.split("-")[0]
    # Create task
    task = ee.batch.Export.image.toDrive(
        image=img[band],
        region=roi,
        description=band + "_" + region,
        folder=region + "/" + tipe,
        fileNamePrefix=time + "-" + band,
        fileFormat='GeoTIFF',
        scale=0.04,
        crs='EPSG:4326'
    )
    task.start()

    # Wait for respond
    task_status = task.status()
    while task_status['state'] in ['READY', 'RUNNING']:
        task_status = task.status()

    # Check if successful
    if task_status['state'] == 'COMPLETED':
        print("{} image export to drive success!".format(band))
    else:
        print('{} image export to drive failed!'.format(band))

### 1-bpm24

In [34]:
# # Karet
# export_to_drive(karet_1bpm24, roi_1bpm24, "1-bpm24", "karet")

In [35]:
# # Gulma
# export_to_drive(gulma_1bpm24, roi_1bpm24, "1-bpm24", "gulma")

In [50]:
export_to_drive1(karet_1bpm24, roi_1bpm24, "1-bpm24", "karet", "re")

1-bpm24 karet images being export to drive
------------------------------------------
re image export to drive success!


### 1-gt1

In [36]:
# # Karet
# export_to_drive(karet_1gt1, roi_1gt1, "1-gt1", "karet")

In [37]:
# # Gulma
# export_to_drive(gulma_1gt1, roi_1gt1, "1-gt1", "gulma")

### 1-rric112

In [42]:
# # Karet
# export_to_drive(karet_1rric112, roi_1rric112, "1-rric112", "karet")

In [43]:
# # Gulma
# export_to_drive(gulma_1rric112, roi_1rric112, "1-rric112", "gulma")

### 2-bpm24

In [44]:
# # Karet
# export_to_drive(karet_2bpm24, roi_2bpm24, "2-bpm24", "karet")

In [45]:
# Gulma
export_to_drive(gulma_2bpm24, roi_2bpm24, "2-bpm24", "gulma")

2-bpm24 gulma images being export to drive
------------------------------------------
rgb image export to drive success!
red image export to drive success!
green image export to drive success!
blue image export to drive success!
nir image export to drive success!
re image export to drive success!


### 2-gt1

In [46]:
# Karet
export_to_drive(karet_2gt1, roi_2gt1, "2-gt1", "karet")

2-gt1 karet images being export to drive
------------------------------------------
rgb image export to drive success!
red image export to drive success!
green image export to drive success!
blue image export to drive success!
nir image export to drive success!
re image export to drive success!


In [47]:
# Gulma
export_to_drive(gulma_2gt1, roi_2gt1, "2-gt1", "gulma")

2-gt1 gulma images being export to drive
------------------------------------------
rgb image export to drive success!
red image export to drive success!
green image export to drive success!
blue image export to drive success!
nir image export to drive success!
re image export to drive success!


### 2-rric112

In [48]:
# Karet
export_to_drive1(karet_2rric112, roi_2rric112, "2-rric112", "karet", "rgb")

2-rric112 karet images being export to drive
------------------------------------------
rgb image export to drive success!


In [49]:
# Gulma
export_to_drive1(gulma_2rric112, roi_2rric112, "2-rric112", "gulma", "rgb")

2-rric112 gulma images being export to drive
------------------------------------------
rgb image export to drive success!
